### 1. Import and load packages 

In [1]:
# Load packages
import pandas as pd
import numpy as np
from datetime import date
import csv
import re
import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from pandas import read_csv
from sklearn.datasets import load_breast_cancer
from datetime import date
import kaplanmeier as km
from sklearn.decomposition import PCA
import pylab as pl
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
%matplotlib inline
from qqman import qqman
from pandas_plink import read_plink
from scipy.stats import ttest_ind
import math
import os
from lifelines import KaplanMeierFitter
from pandas import DataFrame
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
import seaborn as sns; sns.set()
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd
from google.cloud import bigquery
import os
import matplotlib.pyplot as plt
import textwrap
import re
from tqdm import tqdm
%matplotlib inline

In [2]:
pd.set_option("display.max_columns", None)

### 2. Load exposure and outcome cohorts from Big Query 

##### exp_2022 and out_2022

In [3]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'

Set up connection

In [4]:
os.getcwd()

'/home/kbechler/notebooks'

In [5]:
# Use correct path whether you are local or Nero
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/kbechler/adc.json' 

# Set correct Nero project
os.environ['GCLOUD_PROJECT'] = nero_gcp_project
client = bigquery.Client(project = work_project_id)
#client=bigquery.Client();

In [6]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_2022"
exp_df = client.query(sql).to_dataframe()

sql = "select * from som-nero-nigam-starr.kbechler_explore.out_2022"
out_df = client.query(sql).to_dataframe()

/home/kbechler/.conda/envs/env_name/lib/python3.9/site-packages/google/cloud/bigquery/client.py:461: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


### 3. Create CLMBR Patient List 

In [7]:
# Merge outcome and cohort dataframes to create labels column
df = pd.merge(exp_df, out_df, on = 'subject_id', how = 'left')

In [8]:
df.head()

,cohort_definition_id_x,subject_id,cohort_start_date_x,cohort_end_date_x,cohort_definition_id_y,cohort_start_date_y,cohort_end_date_y
0,21,32209409,2011-08-12,2015-07-24,NaN,NaN,NaN
1,21,30924418,2011-09-27,2017-08-30,NaN,NaN,NaN
2,21,31008811,2010-09-16,2019-01-24,22.0,2010-09-16,2019-01-24
3,21,30742058,2011-11-29,2019-10-07,NaN,NaN,NaN
4,21,32400899,2012-08-06,2021-03-02,NaN,NaN,NaN


In [9]:
df.columns = ['exp_id', 'subject_id', 'exp_cohort_start_date', 'exp_cohort_end_date', 'out_id', 
              'out_cohort_start_date', 'out_cohort_end_date']

In [10]:
os.getcwd()

'/home/kbechler/notebooks'

In [11]:
# Bring in patient dataframe with labels for various predictions
patient_df = pd.read_csv('/home/kbechler/patient_df_labels_2022.csv')

In [12]:
patient_df.head()

,cohort_definition_id,subject_id,cohort_start_date,cohort_end_date,person_id_x,ln_diag_date,time_sle_to_ln,index_0,ln_event_0,index_1,ln_event_1,index_2,index_3,index_4,index_5,index_6,index_7,index_8,index_9,ln_event_2,ln_event_3,ln_event_4,ln_event_5,ln_event_6,ln_event_7,ln_event_8,ln_event_9,ln_event_10,ln_event_11,ln_event_12,ln_1year,ln_2years,ln_3years,ln_4years,ln_5years,ln_6years,ln_7years,ln_8years,ln_9years,ln_10years,ln_11years,ln_12years,person_id_y,observation_period_end_DATE,observation_post_sle_obs,event_date,ln_event
0,21,32209409,2011-08-12,2015-07-24,NaN,NaN,NaN,2011-08-12,0,2012-08-11,0.0,2013-08-11,2014-08-11,2015-08-11,2016-08-10,2017-08-10,2018-08-10,2019-08-10,2020-08-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,32209409,2015-07-24,1442,1442.0,0
1,21,30924418,2011-09-27,2017-08-30,NaN,NaN,NaN,2011-09-27,0,2012-09-26,0.0,2013-09-26,2014-09-26,2015-09-26,2016-09-25,2017-09-25,2018-09-25,2019-09-25,2020-09-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,30924418,2017-08-30,2164,2164.0,0
2,21,31008811,2010-09-16,2019-01-24,31008811.0,2016-01-28,1960.0,2010-09-16,0,2011-09-16,0.0,2012-09-15,2013-09-15,2014-09-15,2015-09-15,2016-09-14,2017-09-14,2018-09-14,2019-09-14,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,1,1,1,1,1,1,31008811,2019-01-24,3052,1960.0,1
3,21,30742058,2011-11-29,2019-10-07,NaN,NaN,NaN,2011-11-29,0,2012-11-28,0.0,2013-11-28,2014-11-28,2015-11-28,2016-11-27,2017-11-27,2018-11-27,2019-11-27,2020-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,30742058,2019-10-07,2869,2869.0,0
4,21,32400899,2012-08-06,2021-03-02,NaN,NaN,NaN,2012-08-06,0,2013-08-06,0.0,2014-08-06,2015-08-06,2016-08-05,2017-08-05,2018-08-05,2019-08-05,2020-08-04,2021-08-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,32400899,2021-03-02,3130,3130.0,0


In [13]:
sum(patient_df.ln_5years == 1)

177

##### Set label for if patient develops LN in the next 5 years 

In [14]:
# Merge ln_5years with df
df = pd.merge(df, patient_df[['subject_id', 'ln_5years', 'time_sle_to_ln']], on = 'subject_id', how = 'left')

In [15]:
# Create label column
df['label'] = df.ln_5years

In [16]:
print("Number of positive cases:", sum(df.label == 1))
print("Number of negative cases:", sum(df.label == 0))
print("Total number of patients in exposure cohort:", len(df))

Number of positive cases: 177
Number of negative cases: 2676
Total number of patients in exposure cohort: 2853


In [17]:
df.head()

,exp_id,subject_id,exp_cohort_start_date,exp_cohort_end_date,out_id,out_cohort_start_date,out_cohort_end_date,ln_5years,time_sle_to_ln,label
0,21,32209409,2011-08-12,2015-07-24,NaN,NaN,NaN,0,NaN,0
1,21,30924418,2011-09-27,2017-08-30,NaN,NaN,NaN,0,NaN,0
2,21,31008811,2010-09-16,2019-01-24,22.0,2010-09-16,2019-01-24,0,1960.0,0
3,21,30742058,2011-11-29,2019-10-07,NaN,NaN,NaN,0,NaN,0
4,21,32400899,2012-08-06,2021-03-02,NaN,NaN,NaN,0,NaN,0


In [18]:
# Subset to columns for CLMBR dataframe
df_clmbr = df[['subject_id', 'exp_cohort_start_date', 'label']]

In [19]:
# Rename columns
df_clmbr.columns = ['patient_id', 'date', 'label']

In [20]:
# Change to True/False
df_clmbr['label'] = np.where(df_clmbr.label == 1, True, False)

<ipython-input-20-39c0e5f7b1a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clmbr['label'] = np.where(df_clmbr.label == 1, True, False)


In [21]:
df_clmbr.head()

,patient_id,date,label
0,32209409,2011-08-12,False
1,30924418,2011-09-27,False
2,31008811,2010-09-16,False
3,30742058,2011-11-29,False
4,32400899,2012-08-06,False


### 4. Update outcome cohort in BigQuery to labels with LN within 5 years 

In [22]:
true_labels = df_clmbr[df_clmbr.label == True].patient_id

In [23]:
true_labels = tuple(true_labels)

In [24]:
query = """

CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.out_2022` AS
SELECT *
FROM 
`som-nero-nigam-starr.kbechler_explore.out_2022` out_2022
WHERE
subject_id IN{}""".format(true_labels)

client.query(query).result();

In [25]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.out_2022"
out_2022 = client.query(sql).to_dataframe()

/home/kbechler/.conda/envs/env_name/lib/python3.9/site-packages/google/cloud/bigquery/client.py:461: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [26]:
print(len(out_2022))

177


### 5. Check to see if person's birth is post cohort start date

In [33]:
person_df = pd.read_csv('/home/kbechler/notebooks/data/person.tsv', sep='\t')

In [34]:
person_df.head()

,_gender_name,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_DATETIME,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id,trace_id,unit_id,load_table_id
0,FEMALE,30094567,8532,1971,12,23,1971-12-23 00:00:00,0,0,3473792.0,NaN,NaN,NaN,1 |,8532,|,0,|,0,NaN,shc_,shc_patient
1,FEMALE,31706027,8532,1965,4,30,1965-04-30 00:00:00,0,0,4626316.0,NaN,NaN,NaN,1 |,8532,|,0,|,0,NaN,shc_,shc_patient
2,FEMALE,75819548,8532,1998,9,25,1998-09-25 00:00:00,0,0,1227259.0,NaN,NaN,NaN,1 |,8532,|,0,|,0,NaN,shc_,shc_patient
3,FEMALE,32513341,8532,1946,12,3,1946-12-03 00:00:00,0,0,1806436.0,NaN,NaN,NaN,1 |,8532,|,0,|,0,NaN,shc_,shc_patient
4,FEMALE,31039198,8532,1955,11,27,1955-11-27 00:00:00,8515,38003564,2558798.0,NaN,NaN,NaN,1 |,8532,Asian |,0,Non-Hispanic/Non-Latino |,38003564,NaN,shc_,shc_patient


In [35]:
len(person_df)

2853

In [36]:
# Merge with person_df
qc = pd.merge(df_clmbr, person_df[['person_id', 'birth_DATETIME']], left_on = 'patient_id', right_on = 'person_id', 
             how = 'left')

In [37]:
len(qc)

2853

In [38]:
qc.head()

,patient_id,date,label,person_id,birth_DATETIME
0,32209409,2011-08-12,False,32209409,1982-10-28 00:00:00
1,30924418,2011-09-27,False,30924418,1954-12-23 00:00:00
2,31008811,2010-09-16,False,31008811,1995-01-31 00:00:00
3,30742058,2011-11-29,False,30742058,1968-05-01 00:00:00
4,32400899,2012-08-06,False,32400899,1948-01-11 00:00:00


In [39]:
qc.dtypes

patient_id         int64
date              object
label               bool
person_id          int64
birth_DATETIME    object
dtype: object

In [40]:
qc.date = pd.to_datetime(qc.date)
qc.birth_DATETIME = pd.to_datetime(qc.birth_DATETIME)

In [41]:
qc['start_date_birth_dif'] = (qc.date - qc.birth_DATETIME).dt.days

In [42]:
qc.head()

,patient_id,date,label,person_id,birth_DATETIME,start_date_birth_dif
0,32209409,2011-08-12,False,32209409,1982-10-28,10515
1,30924418,2011-09-27,False,30924418,1954-12-23,20732
2,31008811,2010-09-16,False,31008811,1995-01-31,5707
3,30742058,2011-11-29,False,30742058,1968-05-01,15917
4,32400899,2012-08-06,False,32400899,1948-01-11,23584


In [43]:
qc.start_date_birth_dif.describe()

count     2853.000000
mean     16649.135296
std       6579.567667
min        748.000000
25%      11712.000000
50%      16863.000000
75%      21470.000000
max      32464.000000
Name: start_date_birth_dif, dtype: float64

Confirmed no patient with birth post cohort start date

### 6. Split into train test set

In [78]:
# Need to split to give Ethan test set of patients to exclude when re-training CLMBR

In [107]:
df_clmbr.head()

,patient_id,date,label
0,32209409,2011-08-12,False
1,30924418,2011-09-27,False
2,31008811,2010-09-16,True
3,30742058,2011-11-29,False
4,32400899,2012-08-06,False


In [108]:
X = df_clmbr[['patient_id', 'date']]
y = df_clmbr[['label']]

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size = 0.8)

In [110]:
len(X_train)

2282

In [112]:
len(X_test)

571

In [45]:
# Import train test split
test_split = pd.read_csv('/home/kbechler/test_set.csv')

### 7. Export clmbr_df to csv 

In [46]:
os.getcwd()

'/home/kbechler/notebooks'

In [47]:
# Write to CSV
df_clmbr.to_csv('/home/kbechler/df_clmbr_2022.csv', index = False)

In [48]:
# Write patients to exclude to CSV
# X_test.to_csv('/home/kbechler/test_set.csv', index = False)